In [1519]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import re
from itertools import combinations

In [1520]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1858,tt0120913,75000000,36754634,Titan A.E.,Matt Damon|Bill Pullman|Drew Barrymore|John Le...,Don Bluth|Gary Goldman,"When Earth Ends, The Adventure Begins.",A young man finds out that he holds the key to...,94,Animation|Action|Science Fiction|Family|Adventure,Twentieth Century Fox Film Corporation|David K...,6/16/2000,6.1,2000
1207,tt1588173,35000000,116980662,Warm Bodies,Nicholas Hoult|Teresa Palmer|Analeigh Tipton|R...,Jonathan Levine,Cold body. Warm heart.,After a zombie becomes involved with the girlf...,97,Horror|Comedy|Romance,Summit Entertainment,1/31/2013,6.4,2013
1525,tt0419946,20000000,22165608,The Marine,John Cena|Kelly Carlson|Robert Patrick|Abigail...,John Bonito,War is Fine But The Fight Continues,A group of diamond thieves on the run kidnap t...,92,Action|Adventure|Drama|Thriller,Twentieth Century Fox Film Corporation|Pacific...,10/13/2006,5.0,2006
1793,tt0770772,11000000,13196245,I Think I Love My Wife,Chris Rock|Kerry Washington|Gina Torres|Steve ...,Chris Rock,In marriage no one can hear you scream.,Richard Cooper (Rock) is a married man and fat...,90,Comedy|Romance,Fox Searchlight Pictures,3/16/2007,5.3,2007
776,tt1583420,30000000,36160375,Larry Crowne,Tom Hanks|Julia Roberts|Bryan Cranston|Cedric ...,Tom Hanks,Rediscover life and love,"After losing his job, a middle-aged man reinve...",98,Comedy|Romance|Drama,Universal Pictures|Playtone Productions|Vendom...,6/30/2011,5.6,2011


In [1521]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [1522]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...

def preparation_answer(incomming_data):
    '''
    '''
    movie_name = incomming_data.original_title.iloc[0]
    movie_id = incomming_data.imdb_id.iloc[0]
    print(movie_name, ' (', movie_id , ')', sep='')
    return



def year_detect(release_date):
    '''
    '''
    result = re.search(r'\d{4}', release_date)
    if result:
        return int(result.group(0))
    else:
        return 1970


def month_detect(release_date):
    '''
    '''
    result = re.search(r'^\d\d?', release_date)
    if result:
        month = int(result.group(0))
        if month < 1 or month > 12:
            month = 0
        return month
    else:
        return 0


def series_aggregation_first_column(incoming_data, agg_type='sum', top=5, sort_direction=False,):
    '''
    '''
    result_series = pd.Series(dtype=object)

    for column1, column2 in incoming_data.items():
        
        if agg_type == 'sum':
            agg_parametr = column2
        else:
            agg_parametr = 1
            
        for item in column1.split('|'):
            
            if item in result_series.index:
                result_series[item] += agg_parametr
            else:
                result_series[item] = agg_parametr

    return result_series.sort_values(ascending=sort_direction).head(top)
  


data['profit'] = data.revenue - data.budget


data['release_year'] = data.release_date.apply(year_detect)


data['release_month'] = data.release_date.apply(month_detect)


data['word_count_overview'] = data.overview.apply(lambda s: len(re.findall(r'\w+', s.lower())))


In [1523]:
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,release_month,word_count_overview
1304,tt2083355,17000000,71625195,The Best Man Holiday,Terrence Howard|Harold Perrineau|Morris Chestn...,Malcolm D. Lee,Times change. Friendship doesn't.,When college friends reunite after 15 years ov...,123,Comedy,Universal,11/15/2013,7.0,2013,54625195,11,27
378,tt1130884,80000000,294804195,Shutter Island,Leonardo DiCaprio|Mark Ruffalo|Ben Kingsley|Em...,Martin Scorsese,Someone is missing.,World War II soldier-turned-U.S. Marshal. Tedd...,138,Drama|Thriller|Mystery,Paramount Pictures|Appian Way|Phoenix Pictures...,2/18/2010,7.6,2010,214804195,2,38
1388,tt0398712,30000000,35294470,Assault on Precinct 13,Ethan Hawke|Laurence Fishburne|Gabriel Byrne|M...,Jean-FranÃ§ois Richet,Unite and fight.,"On New Year's Eve, inside a police station tha...",109,Thriller,Rogue Pictures|Why Not Productions|Liaison Fil...,1/19/2005,5.9,2005,5294470,1,42
207,tt1100089,24000000,12096300,Foxcatcher,Steve Carell|Channing Tatum|Mark Ruffalo|Sienn...,Bennett Miller,Based on the shocking true story,The greatest Olympic Wrestling Champion brothe...,134,Drama|Thriller,Likely Story|Media Rights Capital|Annapurna Pi...,5/19/2014,6.5,2014,-11903700,5,34
302,tt1075417,50000000,106303988,Race to Witch Mountain,Dwayne Johnson|AnnaSophia Robb|Alexander Ludwi...,Andy Fickman,The race is on,A taxi driver gets more than he bargained for ...,98,Adventure|Family|Fantasy|Science Fiction|Thriller,Walt Disney Pictures|Gunn Films,3/12/2009,5.4,2009,56303988,3,39


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [1524]:
# 
movie_name = data[data.budget == data.budget.max()]

preparation_answer(movie_name)



Pirates of the Caribbean: On Stranger Tides (tt1298650)


In [1525]:
# +
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650)'

ВАРИАНТ 2

In [1597]:
# ранжируем колонку с бюджетами фильма, последнее значение будет максимальным.
max_budget = data.budget.sort_values().tail(1)

# берем значение индекса у максмального значения и получаем название фильма
data.loc[max_budget.index[0]].original_title

'Pirates of the Caribbean: On Stranger Tides'

# 2. Какой из фильмов самый длительный (в минутах)?

In [1527]:

movie_name = data[data.runtime == data.runtime.max()]

preparation_answer(movie_name)


Gods and Generals (tt0279111)


In [1528]:
# +
answers['2'] = 'Gods and Generals (tt0279111)'

# 3. Какой из фильмов самый короткий (в минутах)?





In [1529]:

movie_name = data[data.runtime == data.runtime.min()]

preparation_answer(movie_name)


Winnie the Pooh (tt1449283)


In [1530]:
# +
answers['3'] = 'Winnie the Pooh (tt1449283)'

# 4. Какова средняя длительность фильмов?


In [1531]:

round(data.runtime.mean())

110

In [1532]:
# +
answers['4'] = 110

# 5. Каково медианное значение длительности фильмов? 

In [1533]:

round(data.runtime.median())

107

In [1534]:
# +
answers['5'] = 107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [1535]:
# лучше код получения столбца profit вынести в Предобработку что в начале

movie_name = data[data.profit == data.profit.max()]

preparation_answer(movie_name)

Avatar (tt0499549)


In [1536]:
# +
answers['6'] = 'Avatar (tt0499549)'

# 7. Какой фильм самый убыточный? 

In [1537]:

movie_name = data[data.profit == data.profit.min()]

preparation_answer(movie_name)

The Lone Ranger (tt1210819)


In [1538]:
# +
answers['7'] = 'The Lone Ranger (tt1210819)'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [1539]:
len(data[data.profit > 0])

1478

In [1540]:
# +
answers['8'] = 1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [1541]:

movie_name = data[data.profit == data.query('release_year == 2008').profit.max()]

preparation_answer(movie_name)

The Dark Knight (tt0468569)


In [1542]:
# +
answers['9'] = 'The Dark Knight (tt0468569)'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [1543]:

movie_name = data[data.profit == data.query('2012 <= release_year <= 2014').profit.min()]

preparation_answer(movie_name)

The Lone Ranger (tt1210819)


In [1544]:
# +
answers['10'] = 'The Lone Ranger (tt1210819)'

# 11. Какого жанра фильмов больше всего?

In [1545]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале

series_aggregation_first_column(data.genres.value_counts(), 'sum', top=1)


Drama    782
dtype: int64

In [1546]:
# +
answers['11'] = 'Drama'

ВАРИАНТ 2

In [1643]:
# создаем пустую коллекцию
genre_counter = Counter()

# делаем выгрузку из дата фрейма и заполняем коллекцию значениями жанров
for genre_all, genre_count in data.genres.value_counts().items():
    for genre in genre_all.split('|'):
        genre_counter[genre] += genre_count

# получаем максимальное значения в словаре
max_count = max(genre_counter.values())

# ищем значение ключа, в котором находится жанр с максимальным количеством
for genre, item in genre_counter.items(): 
    if item == max_count: print(genre)


Drama


# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [1548]:

profitable_film_genre = data[data.profit > 0].groupby(['genres'])['profit'].count()

series_aggregation_first_column(profitable_film_genre, top=1)


Drama    560
dtype: int64

In [1549]:
# или выводим сразу значение жанра для ответа
series_aggregation_first_column(profitable_film_genre).index[0]

'Drama'

In [1550]:
# +
answers['12'] = 'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [1551]:

director_revenue = data.groupby(['director'])['revenue'].sum()

series_aggregation_first_column(director_revenue, top=1)

Peter Jackson    6490593685
dtype: int64

In [1552]:
# +
answers['13'] = 'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [1553]:

director_genres = data[ ['director','genres'] ][data.genres.str.contains('Action', na=False)]

director_genres = director_genres.groupby(['director'])['genres'].count()

series_aggregation_first_column(director_genres, 'sum', top=3)


Robert Rodriguez      9
Paul W.S. Anderson    7
Michael Bay           7
dtype: int64

In [1554]:
# +
answers['14'] = 'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [1555]:

top_actor_2012 = data.query('release_year == 2012').groupby(['cast'])['revenue'].sum()

series_aggregation_first_column( top_actor_2012 ,'sum', top=1)


Chris Hemsworth    2027450773
dtype: int64

In [1556]:
# +
answers['15'] = 'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [1557]:
# высокобюджетные - фильмы, где бюджет выше среднего по данной выборке

top_actor = data[data.budget > data.budget.mean()].groupby(['cast'])['revenue'].count()

series_aggregation_first_column(top_actor ,'sum', top=3)


Matt Damon        18
Adam Sandler      17
Angelina Jolie    16
dtype: int64

In [1585]:
# +
answers['16'] = 'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [1559]:
favorite_genre = data[data.cast.str.contains('Nicolas Cage', na=False)].groupby(['genres'])['cast'].count()

series_aggregation_first_column( favorite_genre ,'sum', top=3)

Action      17
Thriller    15
Drama       12
dtype: int64

In [1560]:
answers['17'] = 'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [1561]:

movie_name = data[data.profit == data[data.production_companies.str.contains('Paramount Pictures')].profit.min()]

preparation_answer(movie_name)

K-19: The Widowmaker (tt0267626)


In [1562]:
answers['18'] = 'K-19: The Widowmaker (tt0267626)'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [1563]:
data.groupby(['release_year'])['revenue'].sum().sort_values().tail(1)

release_year
2015    25449202382
Name: revenue, dtype: int64

In [1564]:
answers['19'] = 2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [1565]:
wb_profit = data[data.production_companies.str.contains('Warner Bros', na=False)]

wb_profit.groupby(['release_year'])['profit'].sum().sort_values(ascending=False).head(1)

release_year
2014    2295464519
Name: profit, dtype: int64

In [1566]:
# +
answers['20'] = 2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [1567]:
# Вариант 1
data.release_month.value_counts().sort_values().tail(1)

9    227
Name: release_month, dtype: int64

In [1568]:
# Вариант 2
data.groupby(['release_month'])['imdb_id'].count().sort_values().tail(1)

release_month
9    227
Name: imdb_id, dtype: int64

In [1569]:
# +
answers['21'] = 'Сентябрь'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [1570]:
# 
len(data.query('release_month in [6, 7, 8]'))

450

In [1571]:
# +
answers['22'] = 450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [1572]:

winter_director = data.query('release_month in [1, 2, 12]').director.value_counts()

series_aggregation_first_column(winter_director,'sum', top=3)

Peter Jackson        7
Steven Soderbergh    6
Clint Eastwood       6
dtype: int64

In [1573]:
# +
answers['23'] = 'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [1574]:

max_len = data.original_title.str.len().max()

data[data.original_title.str.len() == max_len].production_companies.iloc[0].split('|')[1]



'Four By Two Productions'

In [1575]:
# +
answers['24'] = 'Four By Two Productions' 

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [1576]:
# 

word_count_overview_max = data[data.word_count_overview > data.word_count_overview.mean()].word_count_overview.max()

data[data.word_count_overview == word_count_overview_max].production_companies.iloc[0].split('|')[3]


'Midnight Picture Show'

In [1577]:
# +
answers['25'] = 'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [1578]:
# 
count_last_procent = round(0.01 * len(data))

# 
last_procent = data.vote_average.sort_values(ascending=False).head(count_last_procent).index

# 
top_movies = data.loc[last_procent][['original_title','vote_average']]
display(top_movies)

,original_title,vote_average
599,The Dark Knight,8.1
125,The Imitation Game,8.0
9,Inside Out,8.0
118,Interstellar,8.0
34,Room,8.0
1081,The Lord of the Rings: The Return of the King,7.9
138,The Grand Budapest Hotel,7.9
1183,The Wolf of Wall Street,7.9
370,Inception,7.9
119,Guardians of the Galaxy,7.9


In [1579]:
# Проверим, что из варианта №1 все 3 фильма находятся в в выборке 1% лучших фильмов

answer = ['Inside Out', 'The Dark Knight', '12 Years a Slave']

top_movies.query('original_title in @answer')


,original_title,vote_average
599,The Dark Knight,8.1
9,Inside Out,8.0
1191,12 Years a Slave,7.9


In [1580]:
# +
answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave'

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [1662]:
actors_pair = Counter()

actors = data[data.cast.str.contains('|')].cast.unique()

for pair in actors:
    for actor in combinations(sorted(pair.split('|')), 2):
        # print(actor[0], actor[1])
        pair_key = actor[0] + ' & ' + actor[1]
        actors_pair[pair_key] += 1

count_colloboration_max = max(actors_pair.values())

# выведем топ 5 верхних значений по количеству совместных съемок
print(sorted(actors_pair.values(), reverse=True)[:5])

# выведем из словаря только пары актеров, с максимальным количеством совместных съемок
for pair, count_colloboration in actors_pair.items():
    if count_colloboration == count_colloboration_max: 
        print(pair)
    
# 


[8, 8, 8, 6, 6]
Daniel Radcliffe & Emma Watson
Daniel Radcliffe & Rupert Grint
Emma Watson & Rupert Grint


In [1581]:
# +
answers['27'] = 'Daniel Radcliffe & Rupert Grint'

ВАРИАНТ 2

In [1582]:
pair_actors = [  'Johnny Depp & Helena Bonham Carter',
                'Ben Stiller & Owen Wilson',
                'Vin Diesel & Paul Walker',
                'Adam Sandler & Kevin James',
                'Daniel Radcliffe & Rupert Grint']

actor_pair_work = pd.Series(dtype=object)

for pair_actor in pair_actors:
    actor1, actor2 =  pair_actor.split(' & ')
    pair_work_count = data[data.cast.str.contains(actor1) & data.cast.str.contains(actor2)].imdb_id.count()
    #print(pair_work_count, '|', pair_actor)
    actor_pair_work[pair_actor] = pair_work_count

actor_pair_work.sort_values(ascending=False).head(1).index[0]


'Daniel Radcliffe & Rupert Grint'

# Submission

In [1583]:
# в конце можно посмотреть свои ответы к каждому вопросу

# Вопрос № 1 ........ [ + ]
# Вопрос № 2 ........ [ + ]
# Вопрос № 3 ........ [ + ]
# Вопрос № 4 ........ [ + ]
# Вопрос № 5 ........ [ + ]
# Вопрос № 6 ........ [ + ]
# Вопрос № 7 ........ [ + ]
# Вопрос № 8 ........ [ + ]
# Вопрос № 9 ........ [ + ]
# Вопрос № 10 ....... [ + ]
# Вопрос № 11 ....... [ + ]
# Вопрос № 12 ....... [ + ]
# Вопрос № 13 ....... [ + ]
# Вопрос № 14 ....... [ + ]
# Вопрос № 15 ....... [ + ]
# Вопрос № 16 ....... [ + ]
# Вопрос № 17 ....... [ + ]
# Вопрос № 18 ....... [ + ]
# Вопрос № 19 ....... [ + ]
# Вопрос № 20 ....... [ + ]
# Вопрос № 21 ....... [ + ]
# Вопрос № 22 ....... [ + ]
# Вопрос № 23 ....... [ + ]
# Вопрос № 24 ....... [ + ]
# Вопрос № 25 ....... [ + ]
# Вопрос № 26 ....... [ + ]
# Вопрос № 27 ....... [ + ]

answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': 2015,
 '20': 2014,
 '21': 'Сентябрь',
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe & Rupert Grint'}

In [1584]:
# и убедиться что ни чего не пропустил)
len(answers)

27

### Спасибо за внимание!